# Dados de Entrada
* Selecione "Adicionar ao Drive"
  *   https://tinyurl.com/bd-anac







## Acesso ao Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Setup

## Instalação de pacotes

In [1]:
!apt-get update  > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install findspark pyspark

## Preparação do ambiente

In [2]:
%env PYTHONHASHSEED=1234
%env JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
%env SPARK_HOME=/content/spark-3.5.1-bin-hadoop3

env: PYTHONHASHSEED=1234
env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
env: SPARK_HOME=/content/spark-3.5.1-bin-hadoop3


In [3]:
import findspark
findspark.init("/content/spark-3.5.1-bin-hadoop3")

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

from datetime import datetime

appName = "Big Data ANAC"
master = "local[*]"

spark = SparkSession.builder     \
    .master(master) \
    .appName(appName) \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

## Leitura de Dados

In [5]:
import os

In [6]:
anac_path = "/content/drive/MyDrive/Colab Notebooks/anac/"
flights_path = os.path.join(anac_path, "anac2023.parquet")
flights = spark.read.parquet(flights_path)

In [7]:
flights.printSchema()

root
 |-- sigla_icao_empresa_aerea: string (nullable = true)
 |-- numero_voo: integer (nullable = true)
 |-- codigo_di: integer (nullable = true)
 |-- codigo_tipo_linha: string (nullable = true)
 |-- sigla_icao_aeroporto_origem: string (nullable = true)
 |-- partida_prevista: timestamp (nullable = true)
 |-- partida_real: timestamp (nullable = true)
 |-- sigla_icao_aeroporto_destino: string (nullable = true)
 |-- chegada_prevista: timestamp (nullable = true)
 |-- chegada_real: timestamp (nullable = true)
 |-- situacao_voo: string (nullable = true)



In [8]:
aeroportos_path = os.path.join(anac_path, "aeroportos.parquet")
airports = spark.read.parquet(aeroportos_path)


In [9]:
airports.printSchema()

root
 |-- nome: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- pais: string (nullable = true)
 |-- sigla_iata: string (nullable = true)
 |-- sigla_icao: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)



# Join dos Dataframes


In [10]:
flights.createOrReplaceTempView("flights")
airports.createOrReplaceTempView("airports")

In [11]:
spark.sql("select * from flights limit 10").show()

+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+
|sigla_icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|sigla_icao_aeroporto_origem|   partida_prevista|       partida_real|sigla_icao_aeroporto_destino|   chegada_prevista|       chegada_real|situacao_voo|
+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+
|                     AAL|       905|        0|                I|                       KMIA|2023-01-01 21:45:00|2023-01-01 00:56:00|                        SBGL|2023-01-02 06:20:00|2023-01-01 09:03:00|   REALIZADO|
|                     AAL|       929|        0|                I|                       KMIA|2023-01-01 21:45:00|2023-01-01 21:51:00|   

In [12]:
spark.sql("select * from airports limit 10").show()

+--------------------+------------+----------------+----------+----------+---------+---------+
|                nome|      cidade|            pais|sigla_iata|sigla_icao| latitude|longitude|
+--------------------+------------+----------------+----------+----------+---------+---------+
|      Goroka Airport|      Goroka|Papua New Guinea|       GKA|      AYGA| -6.08169|  145.392|
|      Madang Airport|      Madang|Papua New Guinea|       MAG|      AYMD| -5.20708|  145.789|
|Mount Hagen Kagam...| Mount Hagen|Papua New Guinea|       HGU|      AYMH| -5.82679|  144.296|
|      Nadzab Airport|      Nadzab|Papua New Guinea|       LAE|      AYNZ|-6.569803|146.72598|
|Port Moresby Jack...|Port Moresby|Papua New Guinea|       POM|      AYPY| -9.44338|   147.22|
|Wewak Internation...|       Wewak|Papua New Guinea|       WWK|      AYWK| -3.58383|  143.669|
|  Narsarsuaq Airport|Narssarssuaq|       Greenland|       UAK|      BGBW|  61.1605|  -45.426|
|Godthaab / Nuuk A...|    Godthaab|       Greenlan

In [15]:
flights_infos = spark.sql("""
        select
            f.*
            ,a.nome as nome_origem
            ,a.cidade as cidade_origem
            ,a.pais as pais_origem
            ,a.latitude as latitude_origem
            ,a.longitude as longitude_origem
            ,b.nome as nome_destino
            ,b.cidade as cidade_destino
            ,b.pais as pais_destino
            ,b.latitude as latitude_destino
            ,b.longitude as longitude_destino
        from flights f
        left join airports a
            on f.sigla_icao_aeroporto_origem = a.sigla_icao
        left join airports b
            on f.sigla_icao_aeroporto_destino = b.sigla_icao
""")

In [16]:
flights.count()

983867

In [17]:
flights_infos.count()

983867

In [18]:
flights_infos.show()

+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+--------------------+-----------------+-------------+---------------+----------------+--------------------+-----------------+-------------+----------------+-----------------+
|sigla_icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|sigla_icao_aeroporto_origem|   partida_prevista|       partida_real|sigla_icao_aeroporto_destino|   chegada_prevista|       chegada_real|situacao_voo|         nome_origem|    cidade_origem|  pais_origem|latitude_origem|longitude_origem|        nome_destino|   cidade_destino| pais_destino|latitude_destino|longitude_destino|
+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+-----

In [21]:
flights_infos.createOrReplaceTempView("flights_infos")

In [22]:
flights_infos.select(count(when(isnull(flights_infos.cidade_origem), True))\
                     .alias("null_count")).show()

+----------+
|null_count|
+----------+
|     17095|
+----------+



In [23]:
spark.sql("select count(1) from flights_infos where cidade_origem is null").show()

+--------+
|count(1)|
+--------+
|   17095|
+--------+



In [24]:
spark.sql("select count(1) from flights_infos where cidade_destino is null").show()

+--------+
|count(1)|
+--------+
|   17059|
+--------+



In [25]:
spark.sql("select count(1) from flights_infos where partida_prevista is null").show()

+--------+
|count(1)|
+--------+
|   38129|
+--------+



In [26]:
null_count = flights_infos.select(
    *[
        count(when(isnull(c), True)).alias(c)
        for c in flights_infos.columns
    ]
).first()

for column, count in zip(flights_infos.columns, null_count):
    print(f"Column: {column}, Null count: {count}")


Column: sigla_icao_empresa_aerea, Null count: 0
Column: numero_voo, Null count: 2999
Column: codigo_di, Null count: 1186
Column: codigo_tipo_linha, Null count: 0
Column: sigla_icao_aeroporto_origem, Null count: 0
Column: partida_prevista, Null count: 38129
Column: partida_real, Null count: 57420
Column: sigla_icao_aeroporto_destino, Null count: 0
Column: chegada_prevista, Null count: 38129
Column: chegada_real, Null count: 57420
Column: situacao_voo, Null count: 0
Column: nome_origem, Null count: 17095
Column: cidade_origem, Null count: 17095
Column: pais_origem, Null count: 17095
Column: latitude_origem, Null count: 17095
Column: longitude_origem, Null count: 17095
Column: nome_destino, Null count: 17059
Column: cidade_destino, Null count: 17059
Column: pais_destino, Null count: 17059
Column: latitude_destino, Null count: 17059
Column: longitude_destino, Null count: 17059


# Informações de Voo

In [27]:
from geopy.distance import great_circle

# Entrada: latitude e longitude dos pontos a e b
# Saída: distância em km

@udf(returnType=FloatType())
def distance_in_km(lata, longa, latb, longb):
    origin = (lata, longa)
    destination = (latb, longb)
    distance = great_circle(origin, destination).kilometers
    return distance

In [29]:
flights_infos = flights_infos.withColumn("airports_distance(km)", distance_in_km("latitude_destino",
                                                             "longitude_destino",
                                                             "latitude_origem",
                                                             "longitude_origem"))

In [30]:
flights_infos.show(10)

+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+--------------------+-----------------+-------------+---------------+----------------+--------------------+-----------------+-------------+----------------+-----------------+---------------------+
|sigla_icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|sigla_icao_aeroporto_origem|   partida_prevista|       partida_real|sigla_icao_aeroporto_destino|   chegada_prevista|       chegada_real|situacao_voo|         nome_origem|    cidade_origem|  pais_origem|latitude_origem|longitude_origem|        nome_destino|   cidade_destino| pais_destino|latitude_destino|longitude_destino|airports_distance(km)|
+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+--------------

In [34]:
flights_infos.printSchema()

root
 |-- sigla_icao_empresa_aerea: string (nullable = true)
 |-- numero_voo: integer (nullable = true)
 |-- codigo_di: integer (nullable = true)
 |-- codigo_tipo_linha: string (nullable = true)
 |-- sigla_icao_aeroporto_origem: string (nullable = true)
 |-- partida_prevista: timestamp (nullable = true)
 |-- partida_real: timestamp (nullable = true)
 |-- sigla_icao_aeroporto_destino: string (nullable = true)
 |-- chegada_prevista: timestamp (nullable = true)
 |-- chegada_real: timestamp (nullable = true)
 |-- situacao_voo: string (nullable = true)
 |-- nome_origem: string (nullable = true)
 |-- cidade_origem: string (nullable = true)
 |-- pais_origem: string (nullable = true)
 |-- latitude_origem: float (nullable = true)
 |-- longitude_origem: float (nullable = true)
 |-- nome_destino: string (nullable = true)
 |-- cidade_destino: string (nullable = true)
 |-- pais_destino: string (nullable = true)
 |-- latitude_destino: float (nullable = true)
 |-- longitude_destino: float (nullable =

In [31]:
spark.sql("select * from flights_infos limit 10").show()

+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+--------------------+-----------------+-------------+---------------+----------------+--------------------+-----------------+-------------+----------------+-----------------+
|sigla_icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|sigla_icao_aeroporto_origem|   partida_prevista|       partida_real|sigla_icao_aeroporto_destino|   chegada_prevista|       chegada_real|situacao_voo|         nome_origem|    cidade_origem|  pais_origem|latitude_origem|longitude_origem|        nome_destino|   cidade_destino| pais_destino|latitude_destino|longitude_destino|
+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+-----

In [32]:
flights_infos.createOrReplaceTempView("flights_infos")

In [33]:
spark.sql("select * from flights_infos limit 10").show()

+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+--------------------+-----------------+-------------+---------------+----------------+--------------------+-----------------+-------------+----------------+-----------------+---------------------+
|sigla_icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|sigla_icao_aeroporto_origem|   partida_prevista|       partida_real|sigla_icao_aeroporto_destino|   chegada_prevista|       chegada_real|situacao_voo|         nome_origem|    cidade_origem|  pais_origem|latitude_origem|longitude_origem|        nome_destino|   cidade_destino| pais_destino|latitude_destino|longitude_destino|airports_distance(km)|
+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+--------------

In [35]:
flights_infos = flights_infos.withColumn("tempo_voo", col("chegada_real") - col("partida_real"))

In [41]:
flights_infos = flights_infos.withColumn("tempo_voo(segundos)",
                                         (col("chegada_real").cast("long") - col("partida_real").cast("long")))


In [49]:
flights_infos = flights_infos.withColumn("tempo_voo(min)",
                                         round(col("tempo_voo(segundos)").cast("long") / 60, 3))

flights_infos = flights_infos.withColumn("tempo_voo(horas)",
                                         round(col("tempo_voo(segundos)").cast("long") / 3600, 3))

In [47]:
flights_infos = flights_infos.drop(col("tempo_voo"))

In [50]:
flights_infos.show(10)

+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+--------------------+-----------------+-------------+---------------+----------------+--------------------+-----------------+-------------+----------------+-----------------+---------------------+-------------------+--------------+----------------+
|sigla_icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|sigla_icao_aeroporto_origem|   partida_prevista|       partida_real|sigla_icao_aeroporto_destino|   chegada_prevista|       chegada_real|situacao_voo|         nome_origem|    cidade_origem|  pais_origem|latitude_origem|longitude_origem|        nome_destino|   cidade_destino| pais_destino|latitude_destino|longitude_destino|airports_distance(km)|tempo_voo(segundos)|tempo_voo(min)|tempo_voo(horas)|
+------------------------+----------+---------+-----------------+-------

In [51]:
flights_infos.printSchema()

root
 |-- sigla_icao_empresa_aerea: string (nullable = true)
 |-- numero_voo: integer (nullable = true)
 |-- codigo_di: integer (nullable = true)
 |-- codigo_tipo_linha: string (nullable = true)
 |-- sigla_icao_aeroporto_origem: string (nullable = true)
 |-- partida_prevista: timestamp (nullable = true)
 |-- partida_real: timestamp (nullable = true)
 |-- sigla_icao_aeroporto_destino: string (nullable = true)
 |-- chegada_prevista: timestamp (nullable = true)
 |-- chegada_real: timestamp (nullable = true)
 |-- situacao_voo: string (nullable = true)
 |-- nome_origem: string (nullable = true)
 |-- cidade_origem: string (nullable = true)
 |-- pais_origem: string (nullable = true)
 |-- latitude_origem: float (nullable = true)
 |-- longitude_origem: float (nullable = true)
 |-- nome_destino: string (nullable = true)
 |-- cidade_destino: string (nullable = true)
 |-- pais_destino: string (nullable = true)
 |-- latitude_destino: float (nullable = true)
 |-- longitude_destino: float (nullable =

In [52]:
# Velocidade media em km/h
flights_infos = flights_infos.withColumn("vel_media(km/h)", col("airports_distance(km)") / col("tempo_voo(horas)"))

In [58]:
flights_infos = flights_infos.withColumn("diff_chegada_real_prevista(h)",
                                         round((col("chegada_real").cast("long") - col("chegada_prevista").cast("long"))/3600, 3) )

In [59]:
flights_infos = flights_infos.withColumn("diff_saida_real_prevista(h)",
                                         round((col("partida_real").cast("long") - col("partida_prevista").cast("long"))/3600, 3))

In [60]:
flights_infos.show(10)

+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+--------------------+-----------------+-------------+---------------+----------------+--------------------+-----------------+-------------+----------------+-----------------+---------------------+-------------------+--------------+----------------+-----------------+--------------------------+------------------------+-----------------------------+---------------------------+
|sigla_icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|sigla_icao_aeroporto_origem|   partida_prevista|       partida_real|sigla_icao_aeroporto_destino|   chegada_prevista|       chegada_real|situacao_voo|         nome_origem|    cidade_origem|  pais_origem|latitude_origem|longitude_origem|        nome_destino|   cidade_destino| pais_destino|latitude_destino|longitude_destino|airports_distance(k

# Filtros de dados

In [ ]:
# prompt: check brasil or brazil is in the list of unique "pais_origem"

origem_paises = flights_infos.select("pais_origem").distinct().collect()
origem_paises_lista = [pais["pais_origem"] for pais in origem_paises]
"Brasil" in origem_paises_lista or "Brazil" in origem_paises_lista


In [70]:
paises_lst = flights_infos.select("pais_origem").distinct().collect()
paises_lst = [pais["pais_origem"] for pais in paises_lst]

"Brazil" in paises_lst or "Brasil" in paises_lst

True

In [72]:
"Brasil" in paises_lst

False

In [73]:
paises_lst = flights_infos.select("pais_destino").distinct().collect()
paises_lst = [pais["pais_destino"] for pais in paises_lst]

print("Brazil" in paises_lst)
print("Brasil" in paises_lst)

True
False


In [74]:
flights_infos.createOrReplaceTempView("flights_infos")

In [77]:
flights_infos.printSchema()

root
 |-- sigla_icao_empresa_aerea: string (nullable = true)
 |-- numero_voo: integer (nullable = true)
 |-- codigo_di: integer (nullable = true)
 |-- codigo_tipo_linha: string (nullable = true)
 |-- sigla_icao_aeroporto_origem: string (nullable = true)
 |-- partida_prevista: timestamp (nullable = true)
 |-- partida_real: timestamp (nullable = true)
 |-- sigla_icao_aeroporto_destino: string (nullable = true)
 |-- chegada_prevista: timestamp (nullable = true)
 |-- chegada_real: timestamp (nullable = true)
 |-- situacao_voo: string (nullable = true)
 |-- nome_origem: string (nullable = true)
 |-- cidade_origem: string (nullable = true)
 |-- pais_origem: string (nullable = true)
 |-- latitude_origem: float (nullable = true)
 |-- longitude_origem: float (nullable = true)
 |-- nome_destino: string (nullable = true)
 |-- cidade_destino: string (nullable = true)
 |-- pais_destino: string (nullable = true)
 |-- latitude_destino: float (nullable = true)
 |-- longitude_destino: float (nullable =

In [78]:
flights_infos.show()

+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+--------------------+-----------------+-------------+---------------+----------------+--------------------+-----------------+-------------+----------------+-----------------+---------------------+-------------------+--------------+----------------+-----------------+--------------------------+------------------------+-----------------------------+---------------------------+
|sigla_icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|sigla_icao_aeroporto_origem|   partida_prevista|       partida_real|sigla_icao_aeroporto_destino|   chegada_prevista|       chegada_real|situacao_voo|         nome_origem|    cidade_origem|  pais_origem|latitude_origem|longitude_origem|        nome_destino|   cidade_destino| pais_destino|latitude_destino|longitude_destino|airports_distance(k

In [79]:
flights_infos.describe().show()

+-------+------------------------+------------------+-------------------+-----------------+---------------------------+----------------------------+------------+--------------------+-------------+-----------+-------------------+-------------------+--------------------+--------------+------------+-------------------+-------------------+---------------------+-------------------+-----------------+------------------+------------------+--------------------------+------------------------+-----------------------------+---------------------------+
|summary|sigla_icao_empresa_aerea|        numero_voo|          codigo_di|codigo_tipo_linha|sigla_icao_aeroporto_origem|sigla_icao_aeroporto_destino|situacao_voo|         nome_origem|cidade_origem|pais_origem|    latitude_origem|   longitude_origem|        nome_destino|cidade_destino|pais_destino|   latitude_destino|  longitude_destino|airports_distance(km)|tempo_voo(segundos)|   tempo_voo(min)|  tempo_voo(horas)|   vel_media(km/h)|diff_chegada_real_p

In [86]:
# Velocidade media de um aviao de passageiros eh 900km/h
# https://www.terra.com.br/byte/ciencia/por-que-nao-percebemos-a-real-velocidade-de-um-aviao,6d14e86647e38d25a57e6d1850729516ght1mcr8.html

# Informacoes sobre voos no Brasil
# https://www.gov.br/turismo/pt-br/assuntos/noticias/conheca-os-voos-mais-curtos-e-os-mais-longos-realizados-no-brasil


flights_infos = flights_infos.withColumnRenamed("tempo_voo(min)", "tempo_voo_min")\
       .withColumnRenamed("tempo_voo(horas)", "tempo_voo_horas")\
       .withColumnRenamed("vel_media(km/h)", "vel_media_kmh")

flights_infos.createOrReplaceTempView("flights_infos")


In [90]:
flights_infos.select("situacao_voo").distinct().show()

+-------------+
| situacao_voo|
+-------------+
|    CANCELADO|
|NÃO INFORMADO|
|    REALIZADO|
+-------------+



In [91]:
filtered_flights = spark.sql("""
    select
        *
    from flights_infos
    where
        pais_origem = 'Brazil' and
        pais_destino = 'Brazil' and
        cidade_origem <> cidade_destino and
        partida_prevista is not null and
        chegada_prevista is not null and
        partida_real is not null and
        chegada_real is not null and
        tempo_voo_min >= 15 and
        tempo_voo_horas <= 15 and
        vel_media_kmh > 0 and
        vel_media_kmh <= 1200 and
        situacao_voo = 'REALIZADO'
""")

In [92]:
filtered_flights.count()

748484

In [93]:
filtered_flights.show()

+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+--------------------+------------------+-----------+---------------+----------------+--------------------+------------------+------------+----------------+-----------------+---------------------+-------------------+-------------+---------------+------------------+--------------------------+------------------------+-----------------------------+---------------------------+
|sigla_icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|sigla_icao_aeroporto_origem|   partida_prevista|       partida_real|sigla_icao_aeroporto_destino|   chegada_prevista|       chegada_real|situacao_voo|         nome_origem|     cidade_origem|pais_origem|latitude_origem|longitude_origem|        nome_destino|    cidade_destino|pais_destino|latitude_destino|longitude_destino|airports_distance(km)|

# Questões de Análise

## Questão 1: ...

In [96]:
# Cidade com mais voos
filtered_flights.select("cidade_origem")\
    .groupBy("cidade_origem").count().orderBy(desc("count")).show()

+--------------+------+
| cidade_origem| count|
+--------------+------+
|     Sao Paulo|180795|
|Rio De Janeiro| 67078|
|      Campinas| 52226|
|      Brasilia| 51394|
|Belo Horizonte| 45603|
|        Recife| 35630|
|  Porto Alegre| 27694|
|      Salvador| 25649|
|      Curitiba| 24559|
|     Fortaleza| 17934|
|       Goiania| 13836|
|       Vitoria| 13348|
|         Belem| 13346|
| Florianopolis| 13314|
|        Manaus| 10811|
|        Cuiaba| 10498|
|        Maceio|  8532|
|    Navegantes|  8301|
|         Natal|  7886|
|  Porto Seguro|  7263|
+--------------+------+
only showing top 20 rows



## Questão 2: ...

In [111]:
# filtered_flights = filtered_flights.withColumn("rota",
#                                                concat(col("cidade_origem"), lit("_"), col("cidade_destino")))

In [114]:
# Cidades de origem e destino com maior numero de voos
# Cidade com mais voos
filtered_flights.select("cidade_origem", "cidade_destino")\
    .groupBy("cidade_origem", "cidade_destino").count().orderBy(desc("count")).show()
# Grande parte dos voos estao possivelmente vinculados a passageiros que fazem viagem de ida e volta

+--------------+--------------+-----+
| cidade_origem|cidade_destino|count|
+--------------+--------------+-----+
|     Sao Paulo|Rio De Janeiro|26940|
|Rio De Janeiro|     Sao Paulo|26547|
|     Sao Paulo|  Porto Alegre|14506|
|  Porto Alegre|     Sao Paulo|14364|
|Belo Horizonte|     Sao Paulo|13498|
|     Sao Paulo|Belo Horizonte|13371|
|     Sao Paulo|      Brasilia|13290|
|      Brasilia|     Sao Paulo|13224|
|     Sao Paulo|      Curitiba|11248|
|      Curitiba|     Sao Paulo|11203|
|     Sao Paulo|        Recife| 9316|
|        Recife|     Sao Paulo| 9129|
|     Sao Paulo|      Salvador| 8369|
|      Salvador|     Sao Paulo| 8345|
|     Sao Paulo| Florianopolis| 7409|
| Florianopolis|     Sao Paulo| 7346|
|       Goiania|     Sao Paulo| 6456|
|     Sao Paulo|       Goiania| 6368|
|Rio De Janeiro|      Brasilia| 6203|
|      Brasilia|Rio De Janeiro| 6156|
+--------------+--------------+-----+
only showing top 20 rows



## Questão 3: ...

In [98]:
# Ras rotas com maiores atrasos médios na chegada
filtered_flights = filtered_flights.withColumn("rota",
                                               concat(col("cidade_origem"), lit("_"), col("cidade_destino")))

In [99]:
filtered_flights.show(5)

+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+--------------------+------------------+-----------+---------------+----------------+--------------------+------------------+------------+----------------+-----------------+---------------------+-------------------+-------------+---------------+------------------+--------------------------+------------------------+-----------------------------+---------------------------+--------------------+
|sigla_icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|sigla_icao_aeroporto_origem|   partida_prevista|       partida_real|sigla_icao_aeroporto_destino|   chegada_prevista|       chegada_real|situacao_voo|         nome_origem|     cidade_origem|pais_origem|latitude_origem|longitude_origem|        nome_destino|    cidade_destino|pais_destino|latitude_destino|longitude_destino|a

In [116]:
rotas_count = filtered_flights.groupBy("rota").count().filter(col("count") > 100)

In [117]:
filtered_flights.count()

748484

In [118]:
rotas_count.count()

590

In [119]:
rotas_flights_agg = filtered_flights.join(rotas_count, "rota")

In [120]:
rotas_flights_agg.count()

737676

In [123]:
rotas_flights_agg.printSchema()

root
 |-- rota: string (nullable = true)
 |-- sigla_icao_empresa_aerea: string (nullable = true)
 |-- numero_voo: integer (nullable = true)
 |-- codigo_di: integer (nullable = true)
 |-- codigo_tipo_linha: string (nullable = true)
 |-- sigla_icao_aeroporto_origem: string (nullable = true)
 |-- partida_prevista: timestamp (nullable = true)
 |-- partida_real: timestamp (nullable = true)
 |-- sigla_icao_aeroporto_destino: string (nullable = true)
 |-- chegada_prevista: timestamp (nullable = true)
 |-- chegada_real: timestamp (nullable = true)
 |-- situacao_voo: string (nullable = true)
 |-- nome_origem: string (nullable = true)
 |-- cidade_origem: string (nullable = true)
 |-- pais_origem: string (nullable = true)
 |-- latitude_origem: float (nullable = true)
 |-- longitude_origem: float (nullable = true)
 |-- nome_destino: string (nullable = true)
 |-- cidade_destino: string (nullable = true)
 |-- pais_destino: string (nullable = true)
 |-- latitude_destino: float (nullable = true)
 |-- 

In [127]:
rotas_flights_agg.groupBy("rota")\
    .agg(
        avg(col("diff_chegada_real_prevista(h)")).alias("avg_diff_chegada_real_prevista_h"),
        avg(col("diff_chegada_real_prevista")).alias("avg_diff_chegada_real_prevista_s")
    )\
    .orderBy(desc("avg_diff_chegada_real_prevista_s"))\
    .show(10, truncate=False)

+---------------------------+--------------------------------+--------------------------------+
|rota                       |avg_diff_chegada_real_prevista_h|avg_diff_chegada_real_prevista_s|
+---------------------------+--------------------------------+--------------------------------+
|Cuiaba_Campo Grande        |1.201210084033614               |4324.285714285715               |
|Salvador_Morro de Sao Paulo|1.1935126760563386              |4296.6760563380285              |
|Morro de Sao Paulo_Salvador|1.0366450704225343              |3731.830985915493               |
|Belem_Boa Vista            |0.9821456692913384              |3535.748031496063               |
|Belem_Santarem             |0.7859358074222673              |2829.3881644934804              |
|Ribeirao Preto_Brasilia    |0.7384905660377354              |2658.566037735849               |
|Sao Paulo_Campinas         |0.6421021505376344              |2311.6129032258063              |
|Goiania_Campo Grande       |0.568141592

## Questão 4: ...

In [128]:
rotas_flights_agg.printSchema()

root
 |-- rota: string (nullable = true)
 |-- sigla_icao_empresa_aerea: string (nullable = true)
 |-- numero_voo: integer (nullable = true)
 |-- codigo_di: integer (nullable = true)
 |-- codigo_tipo_linha: string (nullable = true)
 |-- sigla_icao_aeroporto_origem: string (nullable = true)
 |-- partida_prevista: timestamp (nullable = true)
 |-- partida_real: timestamp (nullable = true)
 |-- sigla_icao_aeroporto_destino: string (nullable = true)
 |-- chegada_prevista: timestamp (nullable = true)
 |-- chegada_real: timestamp (nullable = true)
 |-- situacao_voo: string (nullable = true)
 |-- nome_origem: string (nullable = true)
 |-- cidade_origem: string (nullable = true)
 |-- pais_origem: string (nullable = true)
 |-- latitude_origem: float (nullable = true)
 |-- longitude_origem: float (nullable = true)
 |-- nome_destino: string (nullable = true)
 |-- cidade_destino: string (nullable = true)
 |-- pais_destino: string (nullable = true)
 |-- latitude_destino: float (nullable = true)
 |-- 

In [138]:
rotas_flights_agg.groupBy("rota").max("vel_media_kmh")\
    .orderBy(desc("max(vel_media_kmh)"))\
    .show(10, truncate=False)
# Validar se continuo tomando o maximo ou deixo a media...talvez pra pergunta a media faça mais sentido

+----------------------+------------------+
|rota                  |max(vel_media_kmh)|
+----------------------+------------------+
|Sao Paulo_Vitoria     |1194.4480763415777|
|Teresina_Brasilia     |1185.759213958986 |
|Campinas_Recife       |1181.3162452239906|
|Brasilia_Curitiba     |1159.6431098439175|
|Campinas_Vitoria      |1155.7644639594264|
|Campinas_Maceio       |1144.3797319472296|
|Sao Paulo_Porto Alegre|1128.5257152970175|
|Sao Paulo_Brasilia    |1114.437799953105 |
|Belem_Manaus          |1113.171418099293 |
|Brasilia_Palmas       |1097.008975935571 |
+----------------------+------------------+
only showing top 10 rows



## Questão 5: ...

In [139]:
filtered_flights.show(10)

+------------------------+----------+---------+-----------------+---------------------------+-------------------+-------------------+----------------------------+-------------------+-------------------+------------+--------------------+------------------+-----------+---------------+----------------+--------------------+------------------+------------+----------------+-----------------+---------------------+-------------------+-------------+---------------+------------------+--------------------------+------------------------+-----------------------------+---------------------------+--------------------+
|sigla_icao_empresa_aerea|numero_voo|codigo_di|codigo_tipo_linha|sigla_icao_aeroporto_origem|   partida_prevista|       partida_real|sigla_icao_aeroporto_destino|   chegada_prevista|       chegada_real|situacao_voo|         nome_origem|     cidade_origem|pais_origem|latitude_origem|longitude_origem|        nome_destino|    cidade_destino|pais_destino|latitude_destino|longitude_destino|a

In [142]:
filtered_flights.select("nome_origem").distinct().show(truncate=False)

+-----------------------------------------------------------------+
|nome_origem                                                      |
+-----------------------------------------------------------------+
|Tefé Airport                                                     |
|Presidente Prudente Airport                                      |
|Umuarama Airport                                                 |
|Maués Airport                                                    |
|Barcelos Airport                                                 |
|Plácido de Castro Airport                                        |
|Nelson Ribeiro Guimarães Airport                                 |
|Hercílio Luz International Airport                               |
|Senadora Eunice Micheles Airport                                 |
|Coronel Horácio de Mattos Airport                                |
|Santa Genoveva Airport                                           |
|João Correa da Rocha Airport                   

In [144]:
# Companhia aérea com menor atraso médio nas saídas
# Vamos usar como referencia os aeroportos usados para partida para contar os mais frequentes,
# porem podemos contar em conjunto os de partida e chegada, se fizer mais sentido

freq_aeroportos = filtered_flights.select("nome_origem")\
                    .groupBy("nome_origem").count().filter(col("count") > 1000)

In [145]:
freq_aeroportos.orderBy(desc("count")).show(10, truncate=False)

+-----------------------------------------------------------------+-----+
|nome_origem                                                      |count|
+-----------------------------------------------------------------+-----+
|Congonhas Airport                                                |90916|
|Guarulhos - Governador André Franco Montoro International Airport|89879|
|Santos Dumont Airport                                            |52368|
|Viracopos International Airport                                  |52226|
|Presidente Juscelino Kubistschek International Airport           |51394|
|Tancredo Neves International Airport                             |45603|
|Guararapes - Gilberto Freyre International Airport               |35630|
|Salgado Filho Airport                                            |27694|
|Deputado Luiz Eduardo Magalhães International Airport            |25649|
|Afonso Pena Airport                                              |24559|
+-------------------------------------

In [156]:
filtered_flights.printSchema()

root
 |-- sigla_icao_empresa_aerea: string (nullable = true)
 |-- numero_voo: integer (nullable = true)
 |-- codigo_di: integer (nullable = true)
 |-- codigo_tipo_linha: string (nullable = true)
 |-- sigla_icao_aeroporto_origem: string (nullable = true)
 |-- partida_prevista: timestamp (nullable = true)
 |-- partida_real: timestamp (nullable = true)
 |-- sigla_icao_aeroporto_destino: string (nullable = true)
 |-- chegada_prevista: timestamp (nullable = true)
 |-- chegada_real: timestamp (nullable = true)
 |-- situacao_voo: string (nullable = true)
 |-- nome_origem: string (nullable = true)
 |-- cidade_origem: string (nullable = true)
 |-- pais_origem: string (nullable = true)
 |-- latitude_origem: float (nullable = true)
 |-- longitude_origem: float (nullable = true)
 |-- nome_destino: string (nullable = true)
 |-- cidade_destino: string (nullable = true)
 |-- pais_destino: string (nullable = true)
 |-- latitude_destino: float (nullable = true)
 |-- longitude_destino: float (nullable =

In [166]:
filtered_flights.filter(col("nome_origem") == "Tancredo Neves International Airport")\
    .groupBy(col("sigla_icao_empresa_aerea"))\
    .agg(
        avg(col("diff_saida_real_prevista(h)")).alias("avg_diff_saida_real_prevista_h"),
        avg(col("diff_saida_real_prevista")).alias("avg_diff_saida_real_prevista_s")
    )\
    .orderBy(desc("avg_diff_saida_real_prevista_s"))\
    .show(10, truncate=False)

+------------------------+------------------------------+------------------------------+
|sigla_icao_empresa_aerea|avg_diff_saida_real_prevista_h|avg_diff_saida_real_prevista_s|
+------------------------+------------------------------+------------------------------+
|RUC                     |3.0                           |10800.0                       |
|SID                     |2.733768264840182             |9841.575342465754             |
|LAE                     |0.767                         |2760.0                        |
|BUY                     |0.333                         |1200.0                        |
|ACN                     |0.09445535714285712           |340.0                         |
|GLO                     |0.09210205336284234           |331.5898192548875             |
|TAM                     |0.07293918918918865           |262.5855855855856             |
|AZU                     |0.06152397552889836           |221.49492463857274            |
+--------------------

## Questão n: ...